<div style="background-color: lightgreen; border-radius: 5px; padding: 10px;">
    <h4>Title</h4>
    <p>check</p>
</div>

In [1]:
import pandas as pd
from itertools import combinations

import time
import warnings
from tqdm.notebook import tqdm
import os

from nltk.tokenize import RegexpTokenizer
from collections import Counter
import re

In [58]:
# !pip install pandas
# !pip install nltk
# !pip install ipywidgets==7.7.0

In [2]:
# Settings that can be changed
BASEFOLDER = '../../../../data/enwiki_extracted/'
# corpus_name = 'city_matrix.csv'
# new_corpus = False

cities_df = pd.read_csv(f'../../input/List_of_cities_300k.csv', delimiter=';')

window_size = 0 # Set to Paragraph

# Change these (three) variables if you want to process more cities!

In [9]:
OUTPUT_PATH = '../../../../data/enwiki_city_pairs/biggest_cities_50/'
updated_nr_of_cities = 55 # by population numbers
current_nr_of_cities = 40

In [10]:
updated_list_of_cities = list(cities_df['Mua_en'][0:updated_nr_of_cities].str.lower())
updated_city_pairs = list(combinations(updated_list_of_cities, 2))
updated_list_of_cities[40:]

# sorted(updated_list_of_cities)

['lille',
 'lodz',
 'marseille',
 'antwerp',
 'bilbao',
 'newcastle',
 'krakow',
 'bochum-herne',
 'thessaloniki',
 'nuremberg',
 'riga',
 'duisburg',
 'dortmund',
 'hanover',
 'zürich']

In [13]:
current_list_of_cities = list(cities_df['Mua_en'][0:current_nr_of_cities].str.lower())
current_city_pairs = list(combinations(current_list_of_cities, 2))

In [14]:
# len(list(combinations([x for x in range(200)], 2))) Shows number of city pairs
print(f'''Adding '{updated_nr_of_cities-current_nr_of_cities}' cities --> From {current_nr_of_cities} to {updated_nr_of_cities} cities.
Adding '{len(updated_city_pairs)-len(current_city_pairs)}' city pairs --> From {len(current_city_pairs)} to {len(updated_city_pairs)} city pairs.
''')

Adding '10' cities --> From 30 to 40 cities.
Adding '345' city pairs --> From 435 to 780 city pairs.



In [17]:
CITY_PAIRS = [item for item in updated_city_pairs if item not in current_city_pairs]
len(CITY_PAIRS)

345

In [18]:
import time

In [44]:
from IPython.display import clear_output
from ipywidgets import Output
from IPython.display import display
out = Output()
display(out)

import sys
tokenizer = RegexpTokenizer('\w+')

def process_wikidump(basefolder=BASEFOLDER,
                     output_dir=OUTPUT_PATH,
                city_pairs = CITY_PAIRS):

    warnings.simplefilter('ignore')
    folders = [f.path for f in os.scandir(basefolder) if f.is_dir()]
    i = 0
    for folder in tqdm(folders, total=len(folders), leave=True, desc='Folders'):
        i+=1
        subfolders = [ f.path for f in os.scandir(folder) if f.is_dir()]
        for subfolder in tqdm(subfolders, total=len(subfolders), leave=False, desc='Sub Folders'):
            if '_V5' in subfolder:
                continue

            for file in os.listdir(subfolder): # tqdm(os.listdir(subfolder), total=len(os.listdir(subfolder)), leave=True, desc='Text Files'):                            
                with open(os.path.join(subfolder, file), "r", encoding="utf-8") as f:
                    process_file(file, f.read(), city_pairs, output_dir) 
                with out:
                    print(subfolder)
                    clear_output(wait=True)

            os.replace(subfolder, subfolder + '_V5')



def process_file(filename, file_content, city_pairs, output_dir):
    articles = file_content.split('</doc>')
    articles = [x.strip() for x in articles if len(x.strip())]
    # time.sleep(1)
    
    for article in articles:
        for paragraph in article.split('\n'):
            tokenized_paragraph = tokenizer.tokenize(paragraph.lower())
        
            word_count = Counter(tokenized_paragraph)
            words = set(word_count.keys())
            
            for city_pair in city_pairs:
                detected_cities = words.intersection(set(city_pair))

                if len(detected_cities) == 2:
                    reg_str = "title=\"" + "(.*?)" + "\""
                    title = re.findall(reg_str, article)

                    reg_str = "id=\"" + "(.*?)" + "\""
                    article_id = re.findall(reg_str, article)
                    
                    with out:
                        print('\r',filename," || ", "title: ", title[0], " || ", "citypair: ", city_pair, end='                                         ')

                    content = f'title="{title[0]}", id={article_id[0]} \n{paragraph} \n\n'

                    textfile = f'{output_dir}{city_pair[0]}_{city_pair[1]}.txt'

                    with open(textfile, 'a+', encoding='utf-16') as f:
                        f.writelines(content)

Output()

In [28]:
import os

os.path.exists("../../../../enwiki_city_pairs")

False

In [45]:
%%time

protection = input(f"Are you sure you want to collect window text of {updated_nr_of_cities} cities? (currently: '{current_nr_of_cities}' cities).\nType 'Yes' to continue.")

if protection == 'Yes':
    print()
    t = 5
    while t:
        print(f"\rcontinuing processing in {t}...", end="")
        t -= 1
        time.sleep(1)
    clear_output()
    process_wikidump(basefolder=BASEFOLDER, output_dir=OUTPUT_PATH, city_pairs=CITY_PAIRS)
else:
    print("\nCanceling Operation...\n")

Folders:   0%|          | 0/62 [00:00<?, ?it/s]

Sub Folders:   0%|          | 0/4 [00:00<?, ?it/s]

Sub Folders:   0%|          | 0/6 [00:00<?, ?it/s]

Sub Folders:   0%|          | 0/6 [00:00<?, ?it/s]

Sub Folders:   0%|          | 0/1 [00:00<?, ?it/s]

...

Sub Folders:   0%|          | 0/1 [00:00<?, ?it/s]

Sub Folders:   0%|          | 0/6 [00:00<?, ?it/s]

Sub Folders:   0%|          | 0/6 [00:00<?, ?it/s]

Sub Folders:   0%|          | 0/6 [00:00<?, ?it/s]

CPU times: total: 2h 40min 48s
Wall time: 3h 12min 19s


<hr>
<hr>
<hr>
<hr>
<hr>
<hr>
<hr>
<hr>
<!-- <br>
<br>
<br>
<br>
<br>
<br> -->
<div style="height: 500px;"></div>